# Dask Array

Материалы:
* Макрушин С.В. Лекция 11: Dask
* https://docs.dask.org/en/latest/array.html
* JESSE C. DANIEL. Data Science with Python and Dask.

## Задачи для совместного разбора

1. Создайте массив размерностью 1000 на 300000, заполненный числами из стандартного нормального распределения. Исследуйте основные характеристики полученного массива.

In [ ]:
import numpy as np

array = np.random.randn(1000, 300000)
print(array.shape)
print(array.size)
print(array.mean())
print(array.min())
print(array.max())

(1000, 300000)
300000000
7.635483359816796e-05
-5.766365106161888
5.606757087408002


2. Посчитайте сумму квадратов элементов массива, созданного в задаче 1. Создайте массив `np.array` такого же размера и сравните скорость решения задачи с использование `da.array` и `np.array`

In [ ]:
import dask.array as da
dask_array = da.from_array(array, chunks=(100, 100))

In [ ]:
%time res1 = np.sum(array**2)

CPU times: user 635 ms, sys: 729 ms, total: 1.36 s
Wall time: 1.37 s


In [ ]:
res2 = da.sum(da.square(dask_array))

In [ ]:
%%time
res2.compute()

CPU times: user 15.7 s, sys: 1.63 s, total: 17.3 s
Wall time: 16.2 s


300019063.6482373

3. Визуализируйте граф вычислений для задачи 12.

In [ ]:
res2 = da.sum(da.square(dask_array[:1]))
res2.visualize()

KeyboardInterrupt: ignored

## Лабораторная работа 11

In [ ]:
import dask.array as da
import h5py
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. Считайте датасет `recipe` из файла `minutes_n_ingredients_full.hdf5` в виде `dask.array`. Укажите аргумент `chunks=(100_000, 3)` при создании массива. Выведите на экран основную информацию о массиве.

In [ ]:
with h5py.File('/content/drive/MyDrive/tobd/minutes_n_ingredients_full.hdf5', 'r') as f:
   data = np.array(f['recipe'])
dask_ar = da.from_array(data, chunks=(100, 3))
dask_ar

dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(100, 3), chunktype=numpy.ndarray>

2. Вычислите среднее значение по каждому столбцу, кроме первого.

In [ ]:
res = da.mean(dask_ar[:, 1:], axis=0)
%time res.compute()

CPU times: user 15.3 s, sys: 1.35 s, total: 16.6 s
Wall time: 17.4 s


array([1004.20805176,    5.4198008 ])

3. Исследуйте, как влияет значение аргумента `chunks` при создании `dask.array` на скорость выполнения операции поиска среднего.

In [ ]:
dask_ar.rechunk(10, 3)
res = da.mean(dask_ar[:, 1:], axis=0)
%time res.compute()


CPU times: user 16.8 s, sys: 1.33 s, total: 18.1 s
Wall time: 19.4 s


array([1004.20805176,    5.4198008 ])

In [ ]:
dask_ar.rechunk(500, 3)
res = da.mean(dask_ar[:, 1:], axis=0)
%time res.compute()

CPU times: user 14 s, sys: 1.19 s, total: 15.2 s
Wall time: 14.6 s


array([1004.20805176,    5.4198008 ])

In [ ]:
dask_ar.rechunk(100000)
res = da.mean(dask_ar[:, 1:], axis=0)
%time res.compute()

CPU times: user 14.5 s, sys: 1.38 s, total: 15.8 s
Wall time: 15.1 s


array([1004.20805176,    5.4198008 ])

In [ ]:
dask_ar.rechunk(500, 3)

dask.array<rechunk-merge, shape=(2231637, 3), dtype=int64, chunksize=(500, 3), chunktype=numpy.ndarray>

4. Выберите рецепты, время выполнения которых меньше медианного значения

In [ ]:
median = da.median(dask_ar[:, 1], axis= 0).compute()
res = dask_ar[dask_ar[:, 1] < median]
res.compute()

array([[1089012,      23,       5],
       [1428572,       0,       5],
       [1400250,      24,       1],
       ...,
       [1029131,      19,       4],
       [1700703,       1,       1],
       [ 713836,       0,       9]])

5. Посчитайте количество каждого из возможных значений кол-ва ингредиентов

In [ ]:
counts = da.bincount(dask_ar[:, 2])
c = counts.compute()
[np.arange(0, len(c))[np.where(c !=0)],c[np.where(c !=0)]]

[array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 40, 43]),
 array([222071, 224158, 229388, 234948, 240720, 244360, 247181, 246747,
        246816,  22430,  19094,  15165,  11640,   8284,   6014,   4145,
          2793,   1913,   1279,    852,    529,    346,    244,    178,
           107,     68,     55,     33,     22,     20,     13,      5,
             4,      3,      4,      1,      2,      1,      1,      2,
             1])]

6. Найдите максимальную продолжительность рецепта. Ограничьте максимальную продолжительность рецептов сверху значением, равному 75% квантилю.

In [ ]:
print(dask_ar[:, 1].max().compute())
p75 = da.percentile(dask_ar[:, 1], q=75).compute()
clipped = dask_ar[:, 1].clip(max=p75).compute()
dask_ar[:, 1] = clipped

2147483647


In [ ]:
dask_ar[:, 1].max().compute()

58

7. Создайте массив `dask.array` из 2 чисел, содержащих ваши предпочтения относительно времени выполнения рецепта и кол-ва ингредиентов. Найдите наиболее похожий (в смысле $L_1$) рецепт из имеющихся в датасете.

In [ ]:
target = da.array([39, 5])

distances = da.absolute(dask_ar[:, [1, 2]] - target).sum(axis=1)

closest_recipe_index = distances.argmin().compute()
np.array(dask_ar[closest_recipe_index])

array([525060,     39,      5])

8. Работая с исходным файлом в формате `hdf5`, реализуйте алгоритм подсчета среднего значения в блочной форме и вычислите с его помощью среднее значение второго столбца в массиве.

Блочный алгоритм вычислений состоит из двух частей:
1. Загрузка фрагмента за фрагментом данных по `blocksize` элементов и проведение вычислений на этим фрагментом
2. Агрегация результатов вычислений на различных фрагментах для получения результата на уровне всего набора данных

Важно: при работе с `h5py` в память загружаются не все элементы, а только те, которые запрашиваются в данный момент

In [ ]:
import h5py

with h5py.File('/content/drive/MyDrive/tobd/minutes_n_ingredients_full.hdf5', 'r') as f:
  blocksize = 100000
  sum = 0
  count = 0
  for i in range(0, f['recipe'].shape[0], blocksize):
      data_block = da.from_array(f['recipe'][i:i+blocksize], chunks=2000)
      sum+= data_block[:, 1:2].sum().compute()
      count += data_block.shape[0]

print(sum/count)


FileNotFoundError: ignored